# Strategy to Develop Free but Profitable Apps in the App Store and Google Play Markets

In this project, we pretend we're working as data analysts for a company that builds Android and iOS mobile apps. We make our apps available on Google Play and the App Store. <br><br>

We only build apps that are free to download and install, and our main source of revenue consists of in-app ads. This means our revenue for any given app is mostly influenced by the number of users who use our app — the more users that see and engage with the ads, the better. Our goal for this project is to analyze data to help our developers understand what type of apps are likely to attract more users. <br> <br>

To do this, we'll need to collect and analyze data about mobile apps available on Google Play and the App Store. <br>
As of September 2018, there were approximately 2 million iOS apps available on the App Store, and 2.1 million Android apps on Google Play. <br> <br>

Collecting data for over 4 million apps requires a significant amount of time and money, so we'll try to analyze a sample of the data instead. To avoid spending resources on collecting new data ourselves, we should first try to see if we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our goals:

- [A data set](https://www.kaggle.com/lava18/google-play-store-apps) containing data about approximately 10,000 Android apps from Google Play; the data was collected in August 2018. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).
- [A data set](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) containing data about approximately 7,000 iOS apps from the App Store; the data was collected in July 2017. You can download the data set directly from [this link](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).


## Summary table
   **1 Initial exploration of the sample data <br>
     2 Deleting a row with missing information <br>
     3 Removing duplicate rows <br>
     4 Removing apps of non-English audience <br>
     5 Extracting the free apps <br>
     6 Most common genres of the apps <br>
     7 Most popular genres of iOS apps <br>
     8 Most popular genres of Android apps <br>
     9 Conclusions**

# 1 Initial exploration of the sample data

In [1]:
# Opening the files
from csv import reader

opened_Apple = open('AppleStore.csv', encoding='utf8')
read_Apple = reader(opened_Apple)
iOS_apps = list(read_Apple)
iOS_header = iOS_apps[0]
iOS_apps =iOS_apps[1:]

opened_Google = open('googleplaystore.csv', encoding='utf8')
read_Google = reader(opened_Google)
android_apps = list(read_Google)
android_header = android_apps[0]
android_apps = android_apps[1:]

To make it easier to explore the two data sets, we'll first write a function named explore_data() that we can use repeatedly to explore rows in a more readable way. We'll also add an option for our function to show the number of rows and columns for any data set.

In [2]:
# Defining a function to explore the data
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
# Printing the header and the first ten rows of iOS and looking at the size of the data
print(iOS_header)
print('\n')
explore_data(iOS_apps, 0, 10, rows_and_columns=True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061

In [4]:
# Printing the header and the first ten rows of android_apps and looking at the size of the data
print(android_header)
print('\n')
explore_data(android_apps, 0, 10, rows_and_columns=True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Eve

We know from the [description of the iOS dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) the meaning of its columns:

1. `id` : App ID

2. `track_name`: App Name

3. `size_bytes`: Size (in Bytes)

4. `currency`: Currency Type

5. `price`: Price amount

6. `rating_count_tot`: User Rating counts (for all versions)

7. `rating_count_ver`: User Rating counts (for current version)

8. `user_rating` : Average User Rating value (for all versions)

9. `userratingver`: Average User Rating value (for current version)

10. `ver` : Latest version code

11. `cont_rating`: Content Rating

12. `prime_genre`: Primary Genre

13. `sup_devices.num`: Number of supporting devices

14. `ipadSc_urls.num`: Number of screenshots showed for display

15. `lang.num`: Number of supported languages

16. `vpp_lic`: Vpp Device Based Licensing Enabled

Hence, the useful columns useful for our analysis are the following ones: `track_name`, `currency`, `price`, `rating_count_tot`, `rating_count_ver`, and `prime_genre`.


The meaning of the columns in the Google Play Store apps dataset is also described [here](https://www.kaggle.com/lava18/google-play-store-apps):
1. `App`: Application name
2. `Category`: Category the app belongs to
3. `Rating`: Overall user rating of the app (as when scraped)
4. `Reviews`: Number of user reviews for the app (as when scraped)
5. `Size`: Size of the app (as when scraped)
6. `Installs`: Number of user downloads/installs for the app (as when scraped)
7. `Type`: Paid or Free
8. `Price`: Price of the app (as when scraped)
9. `Content Rating`: Age group the app is targeted at - Children / Mature 21+ / Adult
10. `Genres`: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres.
11. `Last Updated`: Date when the app was last updated on Play Store (as when scraped)
12. `Current Ver`: Current version of the app available on Play Store (as when scraped)
13. `Android Ver`: Min required Android version (as when scraped)

The useful columns for our analysis are: `App`, `Category`, `Reviews`, `Installs`, `Type`, `Price`, and `Genres`.

# 2 Deleting a row with missing information

[One of the discussions](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) on the Google Play dataset describes an error for row 10472. <br>
Basically it is said that the `Category` value is missing. Let's check by comparing against one row which has all values.

In [5]:
print(android_header)
print('\n')
print(android_apps[10472])
print('\n')
print(android_apps[0])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Effectively the `Category` value is mssing, leading to a column shift for next columns. Let's remove this row and check the dataset size before and after removing the row.

In [6]:
print(len(android_apps))
del android_apps[10472]  # don't run this more than once
print(len(android_apps))

10841
10840


# 3 Removing duplicate rows
It was spotted in the [discussion section](https://www.kaggle.com/lava18/google-play-store-apps/discussion) of the Google Play dataset that some apps have duplicate entries.<br>For instance, Instagram has four entries:

In [7]:
for app in android_apps:
    name = app[0]
    if name == 'Instagram':
        print(app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


All column values are identical but the 4th column, which corresponds to the number of user reviews for the app (as when scraped). <br>
We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. <br>
Intuitively, the higher the number of reviews, the more recent the data should be. Therefore we'll select the rows where the number of reviews is the highest.<br> <br>
But first, let's see how many duplicate apps there are, and let's keep only one row for each of them.

In [8]:
# Creating two lists: one for storing the name of duplicate apps, and one for storing the name of unique apps.
duplicate_apps = []
unique_apps =[]

for app in android_apps:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps: ', len(duplicate_apps))
print('\n')
print('Example of duplicate apps: ', duplicate_apps[:10])


Number of duplicate apps:  1181


Example of duplicate apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


To remove the duplicates, we will:

- Create a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
- Use the information stored in the dictionary and create a new data set, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews).

In [9]:
# Creation of the dictionary
max_reviews = {}

for app in android_apps:
    name = app[0]
    no_reviews = int(app[3])
    if name in max_reviews and max_reviews[name]<no_reviews:
        max_reviews[name] = no_reviews
    elif name not in max_reviews:
        max_reviews[name] = no_reviews
        
print('Expected length: ', len(android_apps)-len(duplicate_apps))
print('Actual length: ', len(max_reviews))

Expected length:  9659
Actual length:  9659


Now, we'll use this dictionary to create a new dataset without the duplicate entries. <br>
As it is possible that some duplicate entries have the same max number of reviews, we'll take the first one. That's the purpose of tha `app_added` variable.

In [10]:
android_apps1 = []
# As some duplicate entries may have the same max number of reviews,
# we want to make sure we add the row to android_apps1 only once
app_added = []
for app in android_apps:
    name = app[0]
    no_reviews = int(app[3])
    if max_reviews[name] == no_reviews and (name not in app_added):
        android_apps1.append(app)
        app_added.append(name)

explore_data(android_apps1, 0, 10, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

# 4 Removing apps of non-English audience
However, if we explore the data long enough, we'll find that both data sets have apps with names that suggest they are not directed toward an English-speaking audience.

In [11]:
print(iOS_apps[813][1])
print(iOS_apps[6731][1])
print('\n')
print(android_apps1[4412][0])
print(android_apps1[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


中国語 AQリスニング
لعبة تقدر تربح DZ


We're not interested in keeping these apps, so we'll remove them. One way to go about this is to remove each app with a name containing a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;), and other symbols (+, *, /).<br> <br>
The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name.

In [12]:
# Creating a function to find if a string contains non-ASCII characters
def isEnglish(s):   
    for c in s:
        if ord(c)>127:
            return False
    return True

print(isEnglish('Instagram'))
print(isEnglish('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(isEnglish('Docs To Go™ Free Office Suite'))
print(isEnglish('Instachat 😜'))


True
False
False
False


Hmmm... This function needs to be revisited, as we can see that emojis and characters like `™` fall outside the ASCII range and have corresponding numbers over 127. <br>
To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English.

In [13]:
# Creating a function to find if a string contains more than 3 non-ASCII characters
def isEnglish(s):
    n_non_English = 0
    for c in s:
        if ord(c)>127:
            n_non_English = n_non_English + 1
    if n_non_English>3:
        return False
    else:
        return True

print(isEnglish('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(isEnglish('Docs To Go™ Free Office Suite'))
print(isEnglish('Instachat 😜'))

False
True
True


It seems to work better! <br>
Now let's filter out non-English apps from both data sets.

In [14]:
iOS_English_apps = []
android_English_apps =[]

for app in iOS_apps:
    name = app[1]
    if isEnglish(name):
        iOS_English_apps.append(app)

for app in android_apps1:
    name = app[0]
    if isEnglish(name):
        android_English_apps.append(app)

In [15]:
explore_data(iOS_English_apps, 0, 10, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061624', '1814', '4.5', '4.0', '6.26', '12+', 'Social Networking', '37', '5', '27', '1']


['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']


['5538347

In [16]:
explore_data(android_English_apps, 0, 10, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

# 5 Extracting the free apps
As we mentioned in the introduction, we only build apps that are free to download and install, and our main source of revenue consists of in-app ads. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

In [17]:
# For iOS app, we'll check if the value in "price" column is higher than 0.0
iOS_apps_free = []
for app in iOS_English_apps:
    price = float(app[4])
    if not price >0.0:
        iOS_apps_free.append(app)

explore_data(iOS_apps_free, 0, 10, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']


['429047995', 'Pinterest', '74778624', 'USD', '0.0', '1061624', '1814', '4.5', '4.0', '6.26', '12+', 'Social Networking', '37', '5', '27', '1']


['282935706', 'Bible', '92774400', 'USD', '0.0', '985920', '5320', '4.5', '5.0', '7.5.1', '4+', 'Reference', '37', '5', '45', '1']


['5538347

The number of English apps we had was 6183 apps. Now we just have 3222 free apps. That means that roughly only half of the English apps were free.

In [18]:
# For android apps, we'll check if the value in "Type" column is "Free"
android_apps_free = []
for app in android_English_apps:
    price = app[7]
    if price == '0':
        android_apps_free.append(app)

explore_data(android_apps_free, 0, 10, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up']


['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up']


['Smoke Effect Photo Maker - Smoke Editor', 'ART_AND_DESIGN', '3.8', '178', '19M', '50,000+

Originally we had 9614 English android apps. Now we have 8863 free apps, which means around 90% of the English apps are free.

# 6 Most common genres of the apps
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful on both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of what are the most common genres for each market. For this, we'll need to build frequency tables for the `prime_genre` column of the App Store data set, and for the `Genres` and `Category` columns of the Google Play data set.<br>
To analyze the frequency tables, we'll build a function (called display_table below) which can help us display the entries in the frequency table in a descending order.

In [19]:
# the freq_table() function takes in two inputs: dataset (which is expected to be a list of lists)
# and index (which is expected to be an integer).
# The function returns the frequency table (as a dictionary) for any column we want.
# The frequencies are expressed as percentages.

def freq_table(dataset, index):
    frequency_occurence = {}
    total = len(dataset)
    for row in dataset:
        a_point = row[index]
        if a_point in frequency_occurence:
            frequency_occurence[a_point] += 1/total*100
        else:
            frequency_occurence[a_point] = 1/total*100
    return frequency_occurence

#The display_table() function :
#Takes in two parameters: dataset and index. dataset is expected to be a list of lists,
#and index is expected to be an integer.
#This function generates a frequency table using the freq_table() function
#It transforms the frequency table into a list of tuples, then sorts the list in a descending order.
#It prints the entries of the frequency table in descending order.

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [20]:
# prime_genre of the Apple app store dataset is the 12th column
display_table(iOS_apps_free, 11)

Games : 58.1626319056464
Entertainment : 7.883302296710134
Photo & Video : 4.965859714463075
Education : 3.6623215394165176
Social Networking : 3.2898820608317867
Shopping : 2.6070763500931133
Utilities : 2.5139664804469306
Sports : 2.1415270018621997
Music : 2.048417132216017
Health & Fitness : 2.0173805090006227
Productivity : 1.7380509000620747
Lifestyle : 1.5828677839851035
News : 1.3345747982619496
Travel : 1.2414649286157668
Finance : 1.1173184357541899
Weather : 0.8690254500310364
Food & Drink : 0.8069522036002481
Reference : 0.558659217877095
Business : 0.5276225946617009
Book : 0.4345127250155184
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665




We can see that among the free English apps, more than a half (58.16%) are games. Entertainment apps are close to 8%, followed by photo and video apps, which are close to 5%. Only 3.66% of the apps are designed for education, followed by social networking apps which amount for 3.29% of the apps in our data set.

The general impression is that App Store (at least the part containing free English apps) is dominated by apps that are designed for fun (games, entertainment, photo and video, social networking, sports, music, etc.), while apps with practical purposes (education, shopping, utilities, productivity, lifestyle, etc.) are more rare. However, the fact that fun apps are the most numerous doesn't also imply that they also have the greatest number of users — the demand might not be the same as the offer.

Let's continue by examining the Genres and Category columns of the Google Play data set (two columns which seem to be related).


In [21]:
# category and genre of the Google Play dataset are the 2nd and 10th columns, respectively
display_table(android_apps_free, 1)

FAMILY : 18.907942238266926
GAME : 9.724729241877363
TOOLS : 8.46119133574016
BUSINESS : 4.591606498194979
LIFESTYLE : 3.90342960288811
PRODUCTIVITY : 3.8921480144404565
FINANCE : 3.7003610108303455
MEDICAL : 3.5311371841155417
SPORTS : 3.3957581227436986
PERSONALIZATION : 3.3167870036101235
COMMUNICATION : 3.2378158844765483
HEALTH_AND_FITNESS : 3.079873646209398
PHOTOGRAPHY : 2.944494584837555
NEWS_AND_MAGAZINES : 2.7978339350180583
SOCIAL : 2.6624548736462152
TRAVEL_AND_LOCAL : 2.335288808664261
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.14350180505415
DATING : 1.861462093862813
VIDEO_PLAYERS : 1.7937725631768928
MAPS_AND_NAVIGATION : 1.398916967509025
FOOD_AND_DRINK : 1.2409747292418778
EDUCATION : 1.1620036101083042
ENTERTAINMENT : 0.9589350180505433
LIBRARIES_AND_DEMO : 0.9363718411552363
AUTO_AND_VEHICLES : 0.9250902527075828
HOUSE_AND_HOME : 0.8235559566787015
WEATHER : 0.8009927797833946
EVENTS : 0.7107400722021667
PARENTING : 0.6543321299638993
ART_AND_DESIGN : 0.6

The landscape seems significantly different on Google Play: there are not that many apps designed for fun, and it seems that a good number of apps are designed for practical purposes (family, tools, business, lifestyle, productivity, etc.). However, if we investigate this further, we can see that the family category (which accounts for almost 19% of the apps) means mostly games for kids (see [here](https://play.google.com/store/apps/category/FAMILY?hl=en_GB)). <br><br>
Even so, practical apps seem to have a better representation on Google Play compared to App Store. This picture is also confirmed by the frequency table we see for the Genres column:

In [22]:
display_table(android_apps_free, 9)

Tools : 8.449909747292507
Entertainment : 6.069494584837599
Education : 5.34747292418777
Business : 4.591606498194979
Productivity : 3.8921480144404565
Lifestyle : 3.8921480144404565
Finance : 3.7003610108303455
Medical : 3.5311371841155417
Sports : 3.46344765342962
Personalization : 3.3167870036101235
Communication : 3.2378158844765483
Action : 3.1024368231047053
Health & Fitness : 3.079873646209398
Photography : 2.944494584837555
News & Magazines : 2.7978339350180583
Social : 2.6624548736462152
Travel & Local : 2.3240072202166075
Shopping : 2.2450361010830324
Books & Reference : 2.14350180505415
Simulation : 2.041967509025268
Dating : 1.861462093862813
Arcade : 1.8501805054151597
Video Players & Editors : 1.771209386281586
Casual : 1.7599277978339327
Maps & Navigation : 1.398916967509025
Food & Drink : 1.2409747292418778
Puzzle : 1.1281588447653441
Racing : 0.9927797833935037
Role Playing : 0.9363718411552363
Libraries & Demo : 0.9363718411552363
Auto & Vehicles : 0.9250902527075828


The difference between the Genres and the Category columns is not crystal clear, but one thing we can notice is that the Genres column is much more granular (it has more categories). We're only looking for the bigger picture at the moment, so we'll only work with the Category column moving forward.

Up to this point, we found that the App Store is dominated by apps designed for fun, while Google Play shows a more balanced landscape of both practical and for-fun apps. Now we'd like to get an idea about the kind of apps that have most users.

# 7 Most popular genres of iOS apps
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the `rating_count_tot` app (user Rating counts for all versions - 6th column).

Let's start with calculating the average number of user ratings per app genre on the App Store.

In [23]:
# Generating a frequency table for the prime_genre column (12th column) to get the unique app genres
frequency_genre = freq_table(iOS_apps_free, 11)

for genre in frequency_genre:
    total = 0 #sum of user ratings
    len_genre = 0 #number of apps specific to each genre
    for app in iOS_apps_free:
        app_genre = app[11]
        if app_genre == genre:
            rating_count_app = float(app[5])
            total += rating_count_app
            len_genre += 1
    average_rating_count = total / len_genre
    print(genre, ': ', '{:.0f}'.format(average_rating_count))           

Social Networking :  71548
Photo & Video :  28442
Games :  22789
Music :  57327
Reference :  74942
Health & Fitness :  23298
Weather :  52280
Utilities :  18684
Travel :  28244
Shopping :  26920
News :  21248
Navigation :  86090
Lifestyle :  16486
Entertainment :  14030
Food & Drink :  33334
Sports :  23009
Book :  39758
Finance :  31468
Education :  7004
Productivity :  21028
Business :  7491
Catalogs :  4004
Medical :  612


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

In [24]:
for app in iOS_apps_free:
    if app[11] == 'Navigation':
        print(app[1], ': ', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic :  345046
Google Maps - Navigation & Transit :  154911
Geocaching® :  12811
CoPilot GPS – Car Navigation & Offline Maps :  3582
ImmobilienScout24: Real Estate Search in Germany :  187
Railway Route Search :  5


The same pattern applies to social networking apps, where the average number is heavily influenced by a few giants like Facebook, Pinterest, Skype, etc. Same applies to music apps, where a few big players like Pandora, Spotify, and Shazam heavily influence the average number.

Our aim is to find popular genres, but navigation, social networking or music apps might seem more popular than they really are. The average number of ratings seem to be skewed by very few apps which have hundreds of thousands of user ratings. We could get a better picture by removing these extremely popular apps for each genre and then rework the averages.

For that, we'll just consider the apps that have between 1000 and 20000 reviews, which means that have a reasonable number of reviews but not to high to remove the top popular apps against which it will be difficult to compete.


In [38]:
# Generating a frequency table for the prime_genre column (12th column) to get the unique app genres
frequency_genre = freq_table(iOS_apps_free, 11)

print('Average number of ratings for apps between 1000 and 20000 reviews')
print('\n')
for genre in frequency_genre:
    total = 0 #sum of user ratings
    len_genre = 0 #number of apps specific to each genre
    for app in iOS_apps_free:
        app_genre = app[11]
        if app_genre == genre:
            rating_count_app = float(app[5])
            if rating_count_app <= 20000 and rating_count_app >1000:
                total += rating_count_app
                len_genre += 1
        if len_genre==0:
            average_rating_count = 0
        else:
            average_rating_count = total / len_genre
    print(genre, ': ', '{:.0f}'.format(average_rating_count), '\n Number of apps between 1000 and 20000 reviews: ', len_genre)

Average number of ratings for apps between 1000 and 20000 reviews


Social Networking :  7199 
 Number of apps between 1000 and 20000 reviews:  37
Photo & Video :  6234 
 Number of apps between 1000 and 20000 reviews:  75
Games :  5466 
 Number of apps between 1000 and 20000 reviews:  606
Music :  6356 
 Number of apps between 1000 and 20000 reviews:  28
Reference :  11386 
 Number of apps between 1000 and 20000 reviews:  7
Health & Fitness :  6242 
 Number of apps between 1000 and 20000 reviews:  29
Weather :  3191 
 Number of apps between 1000 and 20000 reviews:  3
Utilities :  4848 
 Number of apps between 1000 and 20000 reviews:  25
Travel :  4387 
 Number of apps between 1000 and 20000 reviews:  9
Shopping :  8124 
 Number of apps between 1000 and 20000 reviews:  37
News :  6953 
 Number of apps between 1000 and 20000 reviews:  10
Navigation :  8196 
 Number of apps between 1000 and 20000 reviews:  2
Lifestyle :  5551 
 Number of apps between 1000 and 20000 reviews:  21
Entertainm

Let's focus on the apps that have >5000 reviews. <br>
If the number of apps is very high (like `Games` genre), it suggests that the market may be saturated. It is very low (like `Food & Drink` genre), it could be interesting to come into the corresponding market, but it may be more difficult to innovate as not so many people are joining this kind of market. <br>
Therefore we'll fisrt pay attention to the genres that have a middle number of reviews (let's say between 20 and 100). For example let's take a look at `Education` genre, which has almost 6000 reviews and contains 42 apps.

In [40]:
for app in iOS_apps_free:
    if app[11] == 'Education' and float(app[5])<= 20000 and float(app[5])> 1000:
        print(app[1], ':', app[5])

Canvas by Instructure : 19981
ABCmouse.com - Early Learning Academy : 18749
Quizlet: Study Flashcards, Languages & Vocabulary : 16683
Photomath - Camera Calculator : 16523
iTunes U : 15801
Blackboard Mobile Learn™ : 13567
Star Chart : 13482
Remind: Fast, Efficient School Messaging : 9796
PBS KIDS Video : 8651
Toca Kitchen Monsters : 8062
Toca Hair Salon - Christmas Gift : 8049
Edmodo : 7197
Prodigy Math Game : 6683
Epic! - Unlimited Books for Kids : 6676
ChineseSkill -Learn Mandarin Chinese Language Free : 6077
Google Classroom : 5942
TED : 5782
Khan Academy: you can learn anything : 5459
Got It - Homework Help Math, Chem, Physics Solver : 4903
PowerSchool Mobile : 4547
SkyView® Free - Explore the Universe : 4188
Hopscotch : 4057
IXL - Math and English : 3546
Simply Piano by JoyTunes - Learn & play piano : 2925
Kids A-Z : 2887
Infinite Campus Mobile Portal : 2286
PlayKids - Educational Cartoons and Games for Kids : 2196
Memorado Brain Training for Memory & Mindfulness : 2067
Bookshelf 

There is a potential opportunity in this market. It would be interesting to think of a teaching subject not generally covered by these apps (like teaching the use of advanced features of Microsoft Office package?).

Now let's analyze the Google Play market a bit.

# 8 Most popular genres of Android apps
We have data about the number of installs for the Google Play market, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [27]:
display_table(android_apps_free,5) #frequency table of the 'Installs' column

1,000,000+ : 15.726534296029072
100,000+ : 11.552346570397244
10,000,000+ : 10.548285198556075
10,000+ : 10.198555956678813
1,000+ : 8.393501805054239
100+ : 6.915613718411619
5,000,000+ : 6.82536101083039
500,000+ : 5.561823104693188
50,000+ : 4.772111913357437
5,000+ : 4.512635379061404
10+ : 3.5424187725631953
500+ : 3.249097472924202
50,000,000+ : 2.3014440433213004
100,000,000+ : 2.1322202166064965
50+ : 1.9178700361010799
5+ : 0.7897111913357411
1+ : 0.5076714801444041
500,000,000+ : 0.2707581227436822
1,000,000,000+ : 0.22563176895306852
0+ : 0.04512635379061372
0 : 0.01128158844765343


For instance, we don't know whether an app with 100,000+ installs has 100,000 installs, 200,000, or 350,000. However, we don't need very precise data for our purposes — we only want to find out which app genres attract the most users, and we don't need perfect precision with respect to the number of users.

We're going to leave the numbers as they are, which means that we'll consider that an app with 100,000+ installs has 100,000 installs, and an app with 1,000,000+ installs has 1,000,000 installs, and so on. To perform computations, however, we'll need to convert each install number from string to float. This means we need to remove the commas and the plus characters, otherwise the conversion will fail and raise an error.

Now let's calculate the average number of installs per app genre for the Google Play data set.

In [28]:
# Generating a frequency table for the Category column (2nd column) to get the unique app genres
frequency_category = freq_table(android_apps_free, 1)

for category in frequency_category:
    total = 0 #sum of installs specific to each genre
    len_category = 0 #number of apps specific to each genre
    for app in android_apps_free:
        app_category = app[1]
        if app_category == category:
            no_installs = app[5]
            no_installs = no_installs.replace('+','')
            no_installs = no_installs.replace(',','')
            no_installs = float(no_installs)
            total += no_installs
            len_category += 1
    average_installs = total / len_category
    print(category, ': ', '{:.2e}'.format(average_installs))   

ART_AND_DESIGN :  1.99e+06
AUTO_AND_VEHICLES :  6.47e+05
BEAUTY :  5.13e+05
BOOKS_AND_REFERENCE :  8.77e+06
BUSINESS :  1.71e+06
COMICS :  8.18e+05
COMMUNICATION :  3.85e+07
DATING :  8.54e+05
EDUCATION :  1.83e+06
ENTERTAINMENT :  1.16e+07
EVENTS :  2.54e+05
FINANCE :  1.39e+06
FOOD_AND_DRINK :  1.92e+06
HEALTH_AND_FITNESS :  4.19e+06
HOUSE_AND_HOME :  1.33e+06
LIBRARIES_AND_DEMO :  6.39e+05
LIFESTYLE :  1.44e+06
GAME :  1.56e+07
FAMILY :  3.70e+06
MEDICAL :  1.21e+05
SOCIAL :  2.33e+07
SHOPPING :  7.04e+06
PHOTOGRAPHY :  1.78e+07
SPORTS :  3.64e+06
TRAVEL_AND_LOCAL :  1.40e+07
TOOLS :  1.08e+07
PERSONALIZATION :  5.20e+06
PRODUCTIVITY :  1.68e+07
PARENTING :  5.43e+05
WEATHER :  5.07e+06
VIDEO_PLAYERS :  2.47e+07
NEWS_AND_MAGAZINES :  9.55e+06
MAPS_AND_NAVIGATION :  4.06e+06


On average, communication apps have the most installs: 38.5 millions. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs:

In [29]:
def above_100m(category):
    for app in android_apps_free:
        if app[1] == category and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
            print(app[0], ':', app[5])

above_100m('COMMUNICATION')

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times:

In [30]:
def installs_under_X(category, X):
    under_X = []
    for app in android_apps_free:
        n_installs = app[5]
        n_installs = n_installs.replace(',', '')
        n_installs = n_installs.replace('+', '')
        if (app[1] == category) and (float(n_installs) < X):
            under_X.append(float(n_installs))
    average_installs_under_X = sum(under_X) / len(under_X)
    print(category, ': ', int(average_installs_under_X), 'for apps under ', int(X), ' installs') 

installs_under_X('COMMUNICATION',100e6) 

COMMUNICATION :  3603485 for apps under  100000000  installs


We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

We understand then there are two facts that should thoughtfully be considered:
- Categories with very high number of installs may be skewed up by a few appss from IT giants, and it would extremely difficult to compete against them
- Some categories with significant number of installs (let's say above 10M or around it) are already full of apps so the market may be almost saturated

Therefore the strategy will be as follows:
- Look at the categories with high number of installs by first removing the apps with installs>100M
- See if there are opportunities for these categories and join the market

For that, we will reproduce the nested loop we did to get the averae number of installs by category, and a condition in the inner loop not to consider apps with installs>100M.


In [31]:
# Generating a frequency table for the Category column (2nd column) to get the unique app genres
frequency_category = freq_table(android_apps_free, 1)

for category in frequency_category:
    total = 0 #sum of installs specific to each genre
    len_category = 0 #number of apps specific to each genre
    for app in android_apps_free:
        app_category = app[1]
        if app_category == category:
            no_installs = app[5]
            no_installs = no_installs.replace('+','')
            no_installs = no_installs.replace(',','')
            no_installs = float(no_installs)
            if no_installs<100e6 and no_installs>=1e6:
                total += no_installs
                len_category += 1
    average_installs = total / len_category
    print(category, ': \n Average installs:', '{:.2e}'.format(average_installs), '\n Number of apps between 1M and 100M:', len_category)

ART_AND_DESIGN : 
 Average installs: 9.08e+06 
 Number of apps between 1M and 100M: 12
AUTO_AND_VEHICLES : 
 Average installs: 2.67e+06 
 Number of apps between 1M and 100M: 18
BEAUTY : 
 Average installs: 2.30e+06 
 Number of apps between 1M and 100M: 10
BOOKS_AND_REFERENCE : 
 Average installs: 5.31e+06 
 Number of apps between 1M and 100M: 48
BUSINESS : 
 Average installs: 7.30e+06 
 Number of apps between 1M and 100M: 67
COMICS : 
 Average installs: 2.56e+06 
 Number of apps between 1M and 100M: 16
COMMUNICATION : 
 Average installs: 8.30e+06 
 Number of apps between 1M and 100M: 112
DATING : 
 Average installs: 3.91e+06 
 Number of apps between 1M and 100M: 33
EDUCATION : 
 Average installs: 2.97e+06 
 Number of apps between 1M and 100M: 61
ENTERTAINMENT : 
 Average installs: 7.18e+06 
 Number of apps between 1M and 100M: 68
EVENTS : 
 Average installs: 2.33e+06 
 Number of apps between 1M and 100M: 6
FINANCE : 
 Average installs: 3.72e+06 
 Number of apps between 1M and 100M: 92


We can observe that, by removing apps with installs above 100 millions, there is a balance between a good couple of categories which have more than 1 million installs, like `ART AND DESIGN`, `BOOKS AND REFERENCE`, `COMMUNICATION`, `EDUCATION`, `ENTERTAINMENT`, `PERSONALIZATION`, and more.

We'll first concentrate on the genres that have around 100 apps or less, to avoid too high competitive markets.
Let's have a look for example at `EDUCATION` (1.8M installs) and `ART_AND_DESIGN` (2M installs).

In [32]:
no_apps = 0
for app in android_apps_free:
    if app[1] == 'EDUCATION' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])
        no_apps += 1
print('\n')
print('Number of apps above 1 million installs in EDUCATION genre: ', no_apps)

Khan Academy : 5,000,000+
Learn Spanish - Español : 1,000,000+
English for beginners : 1,000,000+
Mermaids : 5,000,000+
Learn Japanese, Korean, Chinese Offline & Free : 1,000,000+
Cars Coloring Pages : 1,000,000+
Math Tricks : 10,000,000+
Learn English Words Free : 5,000,000+
English speaking texts : 1,000,000+
Thai Handwriting : 1,000,000+
THAI DICT 2018 : 1,000,000+
Kanji test · Han search Kanji training (free version) : 1,000,000+
Free intellectual training game application | : 1,000,000+
ABC Preschool Free : 5,000,000+
PINKFONG Baby Shark : 1,000,000+
Lumosity: #1 Brain Games & Cognitive Training App : 10,000,000+
English with Lingualeo : 5,000,000+
Quizlet: Learn Languages & Vocab with Flashcards : 10,000,000+
Udemy - Online Courses : 1,000,000+
Coursera: Online courses : 5,000,000+
edX - Online Courses by Harvard, MIT & more : 1,000,000+
Elevate - Brain Training Games : 5,000,000+
NeuroNation - Focus and Brain Training : 5,000,000+
Memorado - Brain Games : 1,000,000+
Lynda - Onli

The education genre seems to be interesting. We can't identify apps crushing their competitors. <br>
Moreover, the themes of learning look various: foreign languages, science, programming language, and apps with generic teaching (like Coursera or Udemy). <br>
Apparently there is a potential opportunity in this market. For example, we may create an app to teach advanced features of some basic IT tools like Microsoft Apps (which proves to be very useful for professionals). <br> <br>


# 9 Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that there are opportunities for a couple of genres for both iOS and Android. We just concentrated on the example of the `EDUCATION` genre as an example but many other categories are likely to be profitable as well (like `PRODUCTIVITY`, `REFERENCE`, `NEWS` to name a few).